In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

df=pd.read_csv('Clean15YearChennaiWeather.csv')
df=df.sort_values(['year','month','day_name','day_date','Start_hour'],ascending=[True,True,True,True,True])
df.head(3)

,date,date_id,wind,desc,temp,baro,hum,day,month,day_date,year,day_name,Start_hour,End_hour
508,"Monday 14 September 2009, 00.00 — 06.00",1.252886e+12,5.0,Passing clouds.,30.0,1009.0,82,2009-09-14,9,14,2009,0,00:00,06:00
509,"Monday 14 September 2009, 06.00 — 12.00",1.252908e+12,8.0,Clear.,35.0,1011.0,64,2009-09-14,9,14,2009,0,06:00,12:00
510,"Monday 14 September 2009, 12.00 — 18.00",1.252930e+12,6.0,Scattered clouds.,37.0,1007.0,53,2009-09-14,9,14,2009,0,12:00,18:00


In [2]:
df.columns

Index(['date', 'date_id', 'wind', 'desc', 'temp', 'baro', 'hum', 'day',
       'month', 'day_date', 'year', 'day_name', 'Start_hour', 'End_hour'],
      dtype='object')

In [3]:
df[['wind','temp','baro','hum']].corr()

,wind,temp,baro,hum
wind,1.000000,0.547964,0.267167,-0.162249
temp,0.547964,1.000000,-0.216422,-0.829084
baro,0.267167,-0.216422,1.000000,0.726712
hum,-0.162249,-0.829084,0.726712,1.000000


In [4]:
df.isna().sum()

date           0
date_id        0
wind           0
desc           0
temp          30
baro           0
hum            0
day            0
month          0
day_date       0
year           0
day_name       0
Start_hour     0
End_hour       0
dtype: int64

In [5]:
from sklearn.impute import KNNImputer

temp_reshaped = df['temp'].values.reshape(-1, 1)

imputer = KNNImputer(n_neighbors=2)

temp_imputed = imputer.fit_transform(temp_reshaped)

df['temp'] = temp_imputed


In [6]:
train_features=['date_id', 'wind', 'desc', 'baro', 'hum', 'day',
       'month', 'day_date', 'year', 'day_name', 'Start_hour', 'End_hour']
test_features=['temp']


from sklearn_pandas import DataFrameMapper

df['desc']= df['desc'].fillna('missing', inplace=True)
df['desc'] = df['desc'].astype(str)

mapper = DataFrameMapper([
    (['month','wind','baro', 'hum'], None),
    ('desc',CountVectorizer(binary=True, ngram_range=(1, 2)))
])

X=mapper.fit_transform(df)
print(X)

[[9.000e+00 5.000e+00 1.009e+03 8.200e+01 1.000e+00]
 [9.000e+00 8.000e+00 1.011e+03 6.400e+01 1.000e+00]
 [9.000e+00 6.000e+00 1.007e+03 5.300e+01 1.000e+00]
 ...
 [7.000e+00 1.700e+01 1.000e+03 5.700e+01 1.000e+00]
 [7.000e+00 1.700e+01 1.000e+03 5.700e+01 1.000e+00]
 [7.000e+00 1.700e+01 1.000e+03 5.700e+01 1.000e+00]]


In [7]:
x = X.copy()
y = df[test_features].copy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=50)

In [8]:
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler(feature_range=(0,1))
X_train = ms.fit_transform(X_train)
X_test = ms.transform(X_test)

LinearRegression

In [9]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm

model=LinearRegression()
model.fit(X_train,y_train)
prediction = model.predict(X_test)
print('Variance score: %.2f' % model.score(X_test, y_test))
print('different: %.2f' % np.mean(np.absolute(prediction-y_test)))
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, prediction), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, prediction), 2)) 

Variance score: 0.73
different: 1.44
Mean absolute error = 1.44
Mean squared error = 3.63


Decision Tree Regression

In [10]:
from sklearn.tree import DecisionTreeRegressor
dtree=DecisionTreeRegressor(random_state=0)
dtree.fit(X_train,y_train)
prediction2=dtree.predict(X_test)
print('Variance score: %.2f' % dtree.score(X_test, y_test))
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, prediction2), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, prediction2), 2)) 

Variance score: 0.83
Mean absolute error = 0.76
Mean squared error = 2.3


Random Forest Regression

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rnd=RandomForestRegressor(max_depth=90,random_state=0,n_estimators=100)
rnd.fit(X_train,y_train)
prediction3=rnd.predict(X_test)
print('Variance score: %.2f' % rnd.score(X_test, y_test))
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, prediction3), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, prediction3), 2)) 
     

Variance score: 0.90
Mean absolute error = 0.62
Mean squared error = 1.3


In [12]:
# Import necessary libraries
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# SVM Regression
print("Support Vector Machine Regression")

# Scale the features for SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the SVM model
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train.values.ravel())

# Make predictions
svm_predictions = svm_model.predict(X_test_scaled)

# Evaluate the model
print('Variance score: %.2f' % svm_model.score(X_test_scaled, y_test))
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, svm_predictions), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, svm_predictions), 2))




Support Vector Machine Regression
Variance score: 0.86
Mean absolute error = 1.03
Mean squared error = 1.9


In [13]:
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Create and compile the LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(1, X_train.shape[1])),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
history = lstm_model.fit(X_train_lstm, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions
lstm_predictions = lstm_model.predict(X_test_lstm)

# Evaluate the model
mse = sm.mean_squared_error(y_test, lstm_predictions)
mae = sm.mean_absolute_error(y_test, lstm_predictions)
r2 = r2_score(y_test, lstm_predictions)

print('R-squared score:', round(r2, 2))
print("Mean absolute error =", round(mae, 2))
print("Mean squared error =", round(mse, 2))

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
R-squared score: -0.37
Mean absolute error = 3.31
Mean squared error = 18.32


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Assuming X and y are your features and target variables

# Normalize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape input to be [samples, time steps, features]
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

def create_model(units=50, dropout_rate=0.2, learning_rate=0.001):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units, activation='relu', input_shape=(1, X_train.shape[1]), return_sequences=True),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LSTM(units, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model

# Define hyperparameters to try
units_list = [32, 64, 128]
dropout_rates = [0.1, 0.2, 0.3]
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]

best_mse = float('inf')
best_params = {}

# Manual hyperparameter search
for units in units_list:
    for dropout_rate in dropout_rates:
        for learning_rate in learning_rates:
            for batch_size in batch_sizes:
                model = create_model(units, dropout_rate, learning_rate)
                
                early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
                
                history = model.fit(
                    X_train_reshaped, y_train,
                    epochs=100,  # Reduced for quicker iterations
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[early_stopping],
                    verbose=0
                )
                
                mse = model.evaluate(X_test_reshaped, y_test, verbose=0)
                
                if mse < best_mse:
                    best_mse = mse
                    best_params = {
                        'units': units,
                        'dropout_rate': dropout_rate,
                        'learning_rate': learning_rate,
                        'batch_size': batch_size
                    }

print("Best parameters:", best_params)

# Train final model with best parameters
final_model = create_model(
    units=best_params['units'],
    dropout_rate=best_params['dropout_rate'],
    learning_rate=best_params['learning_rate']
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = final_model.fit(
    X_train_reshaped, y_train,
    epochs=500,
    batch_size=best_params['batch_size'],
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
lstm_predictions = final_model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, lstm_predictions)
r2 = r2_score(y_test, lstm_predictions)

print('R-squared score:', round(r2, 2))
print("Mean squared error =", round(mse, 2))

